In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ast import literal_eval
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
df_diff_dep = pd.read_csv('csv/diff_dep_class_enrollment_cnt.csv')
df_class_codes = pd.read_csv('csv/class_codes.csv')
df_data = pd.read_csv('csv/data_cleaned.csv', dtype={"std_no" : "string","department_no": "string","department_code": "string","class_department_code": "string", "class_code": "string","class_name": "string","period": "string"})
df_voc_el = pd.read_csv('csv/data_voc_el.csv', dtype={"std_no" : "string","department_no": "string","department_code": "string","class_department_code": "string", "class_code": "string","class_name": "string","period": "string", "cnt": "string"})
df_soc_el = pd.read_csv('csv/data_social_el.csv', dtype={"std_no" : "string","department_no": "string","department_code": "string","class_department_code": "string", "class_code": "string","class_name": "string","period": "string", "cnt": "string"})
df_mandatory = pd.read_csv('csv/data_mandatory.csv', dtype={"std_no" : "string","department_no": "string","department_code": "string","class_department_code": "string", "class_code": "string","class_name": "string","period": "string", "cnt": "string"})

In [3]:
vocational_elective_courses = pd.read_csv("csv/vocational_elective_courses.csv", 
                               dtype={'department_code': 'string'}, 
                               converters={"vocational_elective_courses": literal_eval})

mandatory_courses = pd.read_csv("csv/mandatory_courses.csv",
                                dtype={'department_code': 'string'},
                                converters={"mandatory_courses": literal_eval})

social_elective_courses = pd.read_csv("csv/social_elective_courses.csv",
                                dtype={'department_code': 'string'},
                                converters={"social_elective_courses": literal_eval})

In [4]:
def dep_records_by_code(dep_code, df_in):
    """verilen bölüm kısaltmasının ders kayıtlarını içeren tablo döndürür"""
    """ilk değişken tablo ikinci değişken tablo uzunlugu döndürür"""
    tmp = df_in[df_in.department_code == dep_code].groupby('std_no').class_name.apply(list).reset_index()
    return tmp, tmp.shape[0]

her bir bölümün öğrencilerinin başka bölümlerden aldığı kayıt sayısının çıkartılması

In [7]:
def different_dep_class_enrollment_cnt_2(df_in):
    dep_codes = df_data['department_code'].unique()
    df2 = pd.DataFrame()
    for dep in dep_codes:
        df = df_in[df_in.department_code == dep]
        df = df[df.department_code != df.class_department_code]
        df['to_drop'] = df.apply(lambda row: 
                            row.class_code in social_elective_courses[social_elective_courses.department_code == row.department_code].reset_index().at[0, 'social_elective_courses'] or 
                            row.class_code in vocational_elective_courses[vocational_elective_courses.department_code == row.department_code].reset_index().at[0, 'vocational_elective_courses'] or 
                            row.class_code in mandatory_courses[mandatory_courses.department_code == row.department_code].reset_index().at[0, 'mandatory_courses'], axis=1)
        df = df[df.to_drop == False].reset_index().drop(['index', 'to_drop'], axis=1)

        df = df[['department_no', 'department_code', 'class_department_code', 'class_code', 'class_name']]
        df['cnt'] = df.groupby('class_department_code')['class_department_code'].transform('count')
        df = df[['department_code', 'class_department_code', 'cnt']]
        df = df.drop_duplicates()
        df = df[df['class_department_code'] != 'ITB']
        df = df.sort_values('cnt', ascending=False).reset_index().drop('index',axis=1)
        df2 = pd.concat([df2, df])
    
    df2 = df2.reset_index().drop('index', axis=1)
    return df2


her bir bölüm için ders kayıt sayısının çıkartılması

In [8]:
def class_enrollment_cnt(dep_code, df_in):
    """verilen bölüm kodunun ders kayıtlarının büyükten küçüğe doğru sıralı bir şekilde
    her ders için kaç kayıt olduğunu döndürür"""
    df = df_in[df_in.department_code == dep_code]
    df = df[['department_no', 'department_code', 'class_department_code', 'class_code', 'class_name', 'period']]
    df['cnt'] = df.groupby(['class_code', 'period'])['class_code'].transform('count')
    df = df.drop_duplicates(subset=df.columns.difference(['class_name']))
    df = df.sort_values('cnt', ascending=False).reset_index().drop('index',axis=1)
    return df

class_enrollment_cnt('BLM', df_voc_el).head(10)

,department_no,department_code,class_department_code,class_code,class_name,period,cnt
0,011,BLM,BLM,BLM3810,Biyoenformatiğe Giriş,BAHAR DÖNEMİ,81
1,011,BLM,BLM,BLM3520,Mobil Programlamaya Giriş,BAHAR DÖNEMİ,81
2,011,BLM,BLM,BLM3620,Sayısal İşaret İşleme,BAHAR DÖNEMİ,79
3,011,BLM,BLM,BLM4120,Büyük Veri İşleme Ve Analizi,BAHAR DÖNEMİ,78
4,011,BLM,BLM,BLM4510,Yapay Zeka,BAHAR DÖNEMİ,77
5,011,BLM,BLM,BLM4800,Veri Madenciliğine Giriş,BAHAR DÖNEMİ,76
6,011,BLM,BLM,BLM4800,Veri Madenciliğine Giriş,GÜZ DÖNEMİ,76
7,011,BLM,BLM,BLM4520,Yapay Sinir Ağlarına Giriş,GÜZ DÖNEMİ,74
8,011,BLM,BLM,BLM4900,İleri Ağ Programlama,BAHAR DÖNEMİ,72
9,011,BLM,BLM,BLM4830,Robot Teknolojisine Giriş,BAHAR DÖNEMİ,64


Frequent item sets oluşturmak için OneShot tablosunun oluşturulması. Oneshot tablosu sütun olarak ders kodları, satır olarak da öğrencileri, değer olarak ise satırdaki öğrencinin hangi dersleri alığına göre 1 veya 0 içeren bir tablodur

In [9]:
def oneshot_df(dep_str, df_in):
    """verilen bölümün kayıtlarının oneshot table'ını oluşturur"""
    dep_records, _ = dep_records_by_code(dep_str, df_in)
    dep_records_class_codes_list = dep_records.class_name.to_list()
    te = TransactionEncoder()
    dep_records_oneshot = te.fit(dep_records_class_codes_list).transform(dep_records_class_codes_list)
    dep_records_oneshot_df = pd.DataFrame(dep_records_oneshot, columns=te.columns_)
    
    return dep_records_oneshot_df


oneshot_df('BLM', df_voc_el)

,Ayrık Olay Simulasyonu,Bilgisayar Grafiğine Giriş,Bilgiye Erişim Ve Arama Motorları,Biyoenformatiğe Giriş,Büyük Veri İşleme Ve Analizi,Doğal Dil İşlemeye Giriş,Görüntü İşleme,Mobil Programlamaya Giriş,Robot Teknolojisine Giriş,Sayısal İşaret İşleme,Sistem Programlama,Uzman Sistemlere Giriş,Veri Madenciliğine Giriş,Veritabanı Sistemlerinin Gerçekleştirilmesi,Yapay Sinir Ağlarına Giriş,Yapay Zeka,Yazılım Kalite ve Test Süreci,Yönetim Bilgi Sistemleri,İleri Ağ Programlama,İstatistiksel Veri Analizi
0,False,False,False,False,True,False,False,False,False,True,False,True,True,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False
2,False,False,True,False,False,False,False,False,True,True,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,True,True,False,False
4,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
333,False,False,False,False,False,False,False,False,True,True,True,True,False,False,False,False,False,False,True,False
334,True,True,False,True,False,False,False,False,True,False,False,False,True,True,False,False,False,False,False,False
335,True,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,True,False,False,True


Oneshot tablosu ile frequent item sets oluşturulması

In [17]:
def create_frequent_itemsets(dep_str, df_in):
    """verilen bölümün kayıtlarına göre frequent itemsets oluşturur"""
    dep_oneshot_df = oneshot_df(dep_str, df_in)
    dep_freq_itemsets = apriori(dep_oneshot_df, min_support = 0.01, use_colnames=True)
    return dep_freq_itemsets

create_frequent_itemsets('BLM', df_voc_el)

0.011869436201780416
frozenset({'İleri Ağ Programlama', 'Yapay Sinir Ağlarına Giriş', 'Robot Teknolojisine Giriş', 'Veritabanı Sistemlerinin Gerçekleştirilmesi'})


frequent itemsets ile istenilen bölüm için kural oluşturulması

In [11]:
def create_rules(dep_str, df_in):
    """verilen bölüm ve verilen ders için öneri yapar"""
    dep_freq_itemsets = create_frequent_itemsets(dep_str, df_in)
    rules = association_rules(dep_freq_itemsets, metric="lift", min_threshold=1)
    if len(rules) == 0:
        return rules
    _, cnt = dep_records_by_code(dep_str, df_in)
    rules["cnt"] = rules.apply(lambda row: round(cnt * row["support"]), axis=1)
    ##listeyi stringe çevirme işlemi
    rules["antecedents"] = rules["antecedents"].apply(lambda x: ', '.join(map(str, x)))
    rules["consequents"] = rules["consequents"].apply(lambda x: ', '.join(map(str, x)))
    ##yuvalama işlemleri
    rules['antecedent support'] = rules['antecedent support'].apply(lambda x: round(x, 2))
    rules['consequent support'] = rules['consequent support'].apply(lambda x: round(x, 2))
    rules['support'] = rules['support'].apply(lambda x: round(x, 2))
    rules['confidence'] = rules['confidence'].apply(lambda x: round(x, 2))
    rules['lift'] = rules['lift'].apply(lambda x: round(x, 2))
    return rules

create_rules('BLM', df_voc_el)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,cnt
0,Bilgisayar Grafiğine Giriş,Ayrık Olay Simulasyonu,0.18,0.17,0.04,0.22,1.30,0.008990,1.064461,13
1,Ayrık Olay Simulasyonu,Bilgisayar Grafiğine Giriş,0.17,0.18,0.04,0.23,1.30,0.008990,1.070458,13
2,Biyoenformatiğe Giriş,Ayrık Olay Simulasyonu,0.24,0.17,0.06,0.26,1.56,0.022374,1.125668,21
3,Ayrık Olay Simulasyonu,Biyoenformatiğe Giriş,0.17,0.24,0.06,0.38,1.56,0.022374,1.215430,21
4,Robot Teknolojisine Giriş,Ayrık Olay Simulasyonu,0.19,0.17,0.06,0.31,1.88,0.027789,1.212841,20
...,...,...,...,...,...,...,...,...,...,...
3649,Mobil Programlamaya Giriş,"İleri Ağ Programlama, Veri Madenciliğine Giriş...",0.24,0.01,0.01,0.05,3.33,0.008303,1.036341,4
3650,Büyük Veri İşleme Ve Analizi,"İleri Ağ Programlama, Mobil Programlamaya Giri...",0.23,0.01,0.01,0.05,4.32,0.009122,1.041543,4
3651,İleri Ağ Programlama,"Mobil Programlamaya Giriş, Veri Madenciliğine ...",0.18,0.04,0.01,0.07,1.84,0.005424,1.032068,4
3652,Veri Madenciliğine Giriş,"İleri Ağ Programlama, Mobil Programlamaya Giri...",0.45,0.01,0.01,0.03,1.77,0.005177,1.011789,4


önce support daha sonra confidence'a göre kuralları sıralayarak ilk 10 kuralın elde edilmesi


In [15]:
def rules10(dep_str, df, antecedents=""):
    """bölüm için veya ders için top10 kuralı döndürür\n
    antecedents girilmezse bölüm, girilirse girilen antecedents için oluşturulu\n
    antecedents list şeklinde olmalıdır. örn: ['Veri Madenciliğine Giriş']"""
    df = create_rules(dep_str, df)
    if len(df) == 0:
        df[['Antecedents', 'Consequents', 'Antec. Sup.', 'Conseq. Sup.', 'Support', 'Confidence', 'Count']] = ""
        return df[['Antecedents', 'Consequents', 'Antec. Sup.', 'Conseq. Sup.', 'Support', 'Confidence', 'Count']]
    
    if len(antecedents) > 0:
        df['to_drop'] = df.apply(lambda row: row.antecedents.find(antecedents), axis=1)
        df = df[df['to_drop'] == False].drop('to_drop', axis=1)
    df = df.sort_values('support', ascending=False)
    df = df.head(10)
    df = df.sort_values('confidence', ascending=False)
    df = df.reset_index().drop('index', axis=1)

    df[['Antecedents', 'Consequents', 'Antec. Sup.', 'Conseq. Sup.', 'Support', 'Confidence', 'Count']]  \
    = df[['antecedents', 'consequents', 'antecedent support', 'consequent support', 'support', 'confidence', 'cnt']]
    return df[['Antecedents', 'Consequents', 'Antec. Sup.', 'Conseq. Sup.', 'Support', 'Confidence', 'Count']]

rules10('BLM', df_voc_el)

,Antecedents,Consequents,Antec. Sup.,Conseq. Sup.,Support,Confidence,Count
0,Büyük Veri İşleme Ve Analizi,Veri Madenciliğine Giriş,0.23,0.45,0.15,0.63,49
1,Doğal Dil İşlemeye Giriş,Veri Madenciliğine Giriş,0.19,0.45,0.11,0.57,36
2,Biyoenformatiğe Giriş,Veri Madenciliğine Giriş,0.24,0.45,0.12,0.49,40
3,Mobil Programlamaya Giriş,Veri Madenciliğine Giriş,0.24,0.45,0.12,0.48,39
4,Sayısal İşaret İşleme,Veri Madenciliğine Giriş,0.23,0.45,0.11,0.46,36
5,Veri Madenciliğine Giriş,Büyük Veri İşleme Ve Analizi,0.45,0.23,0.15,0.32,49
6,Veri Madenciliğine Giriş,Mobil Programlamaya Giriş,0.45,0.24,0.12,0.26,39
7,Veri Madenciliğine Giriş,Biyoenformatiğe Giriş,0.45,0.24,0.12,0.26,40
8,Veri Madenciliğine Giriş,Doğal Dil İşlemeye Giriş,0.45,0.19,0.11,0.24,36
9,Veri Madenciliğine Giriş,Sayısal İşaret İşleme,0.45,0.23,0.11,0.24,36


In [ ]:
def pie_inputs_normal(dep_str, df_in, class_str="", cnt_in=10, text_len=20):
    """dep_str: bölüm kısaltması\n
    df_in: df_voc_el/df_soc_el\n
    class_str:class_str"""
    df = class_enrollment_cnt(dep_str, df_in)
    if len(df) < cnt_in:
        cnt = len(df)
    else:
        cnt = cnt_in
    
    colors = ['#046dc8', '#1184a7', '#ea515f', '#b4418e', '#d94a8c', '#15a2a2', '#0450b4', '#6fb1a0', '#fe7434', '#fea802', '#b4418e', '#d94a8c', '#0450b4']
    counts = df.head(cnt)['cnt'].to_list()
    explode = list(np.full(cnt, 0.02))
    labels_ = df.head(cnt)['class_name'].to_list()
    labels = list()
    for i in range(0, cnt):
        if len(df.at[i, 'class_name']) > text_len:
            labels.append(f"{df.at[i, 'class_name'][:text_len]}... [{df_class_codes[df_class_codes.class_name==df.at[i, 'class_name']].reset_index().at[0, 'class_code']}][{df.at[i, 'period'][:1]}]")
        else:
            labels.append(f"{df.at[i, 'class_name']} [{df_class_codes[df_class_codes.class_name==df.at[i, 'class_name']].reset_index().at[0, 'class_code']}][{df.at[i, 'period'][:1]}]")
    
    if class_str!="":
        tmp_df = df[df['class_name'] == class_str]
        if class_str not in labels_:
            labels_.append(class_str)
            for row in tmp_df.iterrows():
                if len(row[1][4]) > text_len:
                    labels.append(f"{row[1][4][:text_len]}... [{df_class_codes[df_class_codes.class_name==row[1][4]].reset_index().at[0, 'class_code']}][{row[1][5][:1]}]")
                else:
                    labels.append(f"{row[1][4]} [{df_class_codes[df_class_codes.class_name==row[1][4]].reset_index().at[0, 'class_code']}][{row[1][5][:1]}]")
                counts.append(row[1][6])
                explode.append(0.2)
        else:
            for row in tmp_df.iterrows():
                try:
                    explode[row[0]] = 0.2
                except:
                    #hata verirse o satırı eklicez
                    if len(row[1][4]) > text_len:
                        labels.append(f"{row[1][4][:text_len]}... [{df_class_codes[df_class_codes.class_name==row[1][4]].reset_index().at[0, 'class_code']}][{row[1][5][:1]}]")
                    else:
                        labels.append(f"{row[1][4]} [{df_class_codes[df_class_codes.class_name==row[1][4]].reset_index().at[0, 'class_code']}][{row[1][5][:1]}]")
                    counts.append(row[1][6])
                    explode.append(0.2)
        print(tmp_df)
    
    if (sum(df['cnt'].to_list()) - sum(counts)) > 0:
        labels.append('Diğerleri')
        counts.append(sum(df['cnt'].to_list()) - sum(counts))
        explode.append(0.02)

    return counts, labels, colors, explode



In [ ]:
def pie_inputs_diff(dep_str, cnt_in=10, text_len=20):
    """dep_str: bölüm kısaltması"""
    df = df_diff_dep[df_diff_dep['department_code'] == dep_str]
    if len(df) < cnt_in:
        cnt = len(df)
    else:
        cnt = cnt_in
    colors = ['#046dc8', '#1184a7', '#ea515f', '#b4418e', '#d94a8c', '#15a2a2', '#0450b4', '#6fb1a0', '#fe7434', '#fea802', '#b4418e', '#d94a8c', '#0450b4']
    
    explode = list(np.full(cnt, 0.02))
    counts = df.head(cnt)['cnt'].to_list()
    labels = list()

    for row in df.head(cnt).iterrows():
        if len(dep_dict[row[1][0]][0]) > text_len:
            labels.append(f"{dep_dict[row[1][1]][0][:text_len]}... [{row[1][1]}]")
        else:
            labels.append(f"{dep_dict[row[1][1]][0]} [{row[1][1]}]")
    
    if (sum(df['cnt'].to_list()) - sum(counts)) > 0:
        labels.append('Diğerleri')
        counts.append(sum(df['cnt'].to_list()) - sum(counts))
        explode.append(0.02)
    
    return counts, labels, colors, explode

In [ ]:
def pie_inputs_rules(df_in, cnt_in=10, text_len=20):
    """df_in: rules10(dep_str, df_voc_el/df_soc_el, ["class_str"])"""
    if len(df_in) < cnt_in:
        cnt = len(df_in)
    else:
        cnt = cnt_in
    colors = ['#046dc8', '#1184a7', '#ea515f', '#b4418e', '#d94a8c', '#15a2a2', '#0450b4', '#6fb1a0', '#fe7434', '#fea802', '#b4418e', '#d94a8c', '#0450b4']
    explode = list(np.full(cnt, 0.02))
    counts = list()  
    labels = list()

    for row in df_in.iterrows():
        if len(row[1][1])>text_len:
            labels.append(f"{row[1][1][:text_len]}... [{df_class_codes[df_class_codes.class_name==row[1][1]].reset_index().at[0, 'class_code']}]")
        else:
            labels.append(f"{row[1][1]} [{df_class_codes[df_class_codes.class_name==row[1][1]].reset_index().at[0, 'class_code']}]")
        counts.append(row[1][6])
    
    return counts, labels, colors, explode

In [ ]:
def draw_pie_with_inputs(counts, labels, colors, explode):

    _, texts, autotexts = plt.pie(counts, labels=labels, startangle=45, autopct=lambda p: '{:.0f}'.format(p/100*sum(counts)), colors=colors, pctdistance=0.80, explode=explode)
    for text in texts:
        text.set_size('xx-small')
        text.set_color(colors[labels.index(text.get_text()) % len(colors)])
    
    for autotext in autotexts:
        autotext.set_size('xx-small')
        autotext.set_color('#113322')
    
    plt.show()